## Text Analysis (with Vader) Women's Clothing Review
### Kiran Pandey April 27, 2019

Function to analyze a "Text" column in a dataframe and calcualte a setiment score


and calculate a sentiment value. Make a new column in the dataframe that will contain the sentiment value for each review.Append a new column for sentiment value of each review in the women's e-commerce clothing reviews.
Source : Download the csv file for  (Links to an external site.)Links to an external site. from Kaggle. 

Input source:  https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews

Output : dataframe of clothing reviews with a sentiment category and sentiment value score.

## libraries used

In [25]:
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize

from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from string import punctuation

## Function definitions 

In [26]:
def clean_str (in_str,dbg=False):
    ''' returns string in lower case and punctuation removed'''
    cln_str = in_str.lower()                                   # change all the words to lowercase
    if dbg: cln_str[0:400]
    translator = cln_str.maketrans('‘’', '  ', punctuation)    # create dictionary to remap punctuation to none, add ‘’ also to list
    cln_str = cln_str.translate(translator)                      # strip all puctuation
    if dbg: cln_str[0:400]      
    return cln_str

In [27]:
def lem_strls (strls,pos):
    ''' return lemmatized string list using chosen part of speech'''
    wnl = WordNetLemmatizer()      # create instance of wordnetlemmatizer
    pos = 'v'
    lem_words = []
    for word in strls:
        lem_words.append(wnl.lemmatize(word, pos))  #lemmatize using lem_opt part-of-speech               
    return  lem_words  

In [28]:
def rev_sent(in_str,lem_opts='',dbg=False):
    ''' compute sentiment polarity of text 
        by default lemmatizes verbs, nouns and adjectives'''

    tkns = word_tokenize(clean_str(in_str))    # tokenize cleaned text
    
    if dbg: rem_tkns = []                     # placeholder for removed words

    cln_tkns = []                             # placeholder for clean words  
    for tkn in tkns:                          # remove filler words
        if tkn not in stopwords.words('english'):  
            cln_tkns.append(tkn)            
        else: 
            if dbg: rem_tkns.append(tkn)        # capture removed words if dbg
      
    lem_tkns = cln_tkns  
    for lem_opt in lem_opts:                    # loop thorough selected parts of speech and lemmtize 
        lems_tkns = lems_strls(lem_tkns,lem_opt)
                   
    final_str = " ".join(lem_tkns)              # convert final word list to a single string   

    sid = SentimentIntensityAnalyzer()          #  create instance of sentiment intensity analyzer
    sid_score = sid.polarity_scores(final_str)  #extract the sentiment value from the dictionary of scores
   
    if dbg:                                     # provide captures diagnostics for debugging
        print ('removed word count = ', len(rem_tkns))   # confirm/validate removed words are appropriate
        fd_rm = FreqDist(rem_tkns)
        print ('no of unique removed words = ', len(rem_tkns))
        fd_rm.most_common(15)

        print ('new word count = ', len(lem_tkns))       # this is the list of lemmatized words
        fd_nw = FreqDist(lemm_a)
        print ('no of unique words = ', len(fd_nw))
        fd_nw.most_common(15)

        for i, tkn in enumerate(lem_tkns):               # list of all new_words and its index 
            print (i, tkn)
        
        print ('sid polarity score = ',sid_score)          
        return sid_score, rem_tkns, lem_tkns
   
    return sid_score['compound'] 

In [29]:
def indx_wrd (in_str, final_str):
    ''' return index of all occurences of fianl clean lemmatized string in original string
    To be completed '''  
    Pass
    return 

In [30]:
#create a function to assign a polarity category to the sentiment
def sentimentCategory(sent_num):
    if sent_num >= 0.2:
        return "positive"
    if sent_num <= -0.2:
        return "negative"
    else:
        return "neutral"

In [31]:
def cat_sent(sent_num,thresh=0.2):
    '''Assign a polarity to a sentiment category
       TO be debugged --- This function currently does not work 
    '''
    cut_points = [-1,-thresh,thresh,1]
    bin_name = ["negative","neutral","positive"]
    print (cut_points)
    print (bin_name)
    temp = pd.cut(sent_num,cut_points,retbins="True",labels=bin_name)
    print (sent_num.shape)
    type(temp)
    print (temp)
    return temp

In [32]:
def exp_df(df,obj="False"):
    '''basic data frame exploration 
    to be completed -- return areas needing futher review'''  
    print ('\nshape: ', df.shape)
    print ('\ncolumns: ',df.dtypes)
    print ('\n',df.describe(include=[np.number]))
    if obj: 
        print ('\n',df.describe(include=[np.object]))
        print ('\n',df.describe(include=['category']))
    return 

### Main program start here

In [33]:
wm_clth_rev = pd.read_csv('../datasets/women_clothing_review.csv', 
            dtype={'Division Name':'category', 'Department Name':'category','Class Name':'category'})    # read csv file 

In [34]:
wm_clth_rev= wm_clth_rev[wm_clth_rev['Review Text'].notnull()]  # drop observations with no reviews
# wm_clth_rev = wm_clth_rev[:200]

In [35]:
exp_df(wm_clth_rev,True)         # explore data


shape:  (22641, 11)

columns:  Unnamed: 0                    int64
Clothing ID                   int64
Age                           int64
Title                        object
Review Text                  object
Rating                        int64
Recommended IND               int64
Positive Feedback Count       int64
Division Name              category
Department Name            category
Class Name                 category
dtype: object

          Unnamed: 0   Clothing ID           Age        Rating  \
count  22641.000000  22641.000000  22641.000000  22641.000000   
mean   11740.849035    919.332362     43.280376      4.183561   
std     6781.957509    202.266874     12.326980      1.115762   
min        0.000000      1.000000     18.000000      1.000000   
25%     5872.000000    861.000000     34.000000      4.000000   
50%    11733.000000    936.000000     41.000000      5.000000   
75%    17621.000000   1078.000000     52.000000      5.000000   
max    23485.000000   1205.000000   

In [36]:
wm_clth_rev['sent_scr']=wm_clth_rev['Review Text'].apply(rev_sent)

In [37]:
#create a new column to hold sentiment category
wm_clth_rev['sent_cat'] = wm_clth_rev['sent_scr'].apply(sentimentCategory)

# repalce with the following function call with desired threshhold parameter (currnelty does not work)
#wm_clth_rev['sent_cat']=wm_clth_rev['sent_scr'].apply(cat_sent,thresh=0.2)

In [38]:
wm_clth_rev.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,sent_scr,sent_cat
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates,0.8991,positive
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses,0.9710,positive
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,0.9062,positive
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,0.9464,positive
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,0.9062,positive


In [39]:
wm_clth_rev['sent_cat'].value_counts()

positive    21329
neutral       749
negative      563
Name: sent_cat, dtype: int64